In [1]:
import time
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import Counter

In [6]:
# Load CSV
df = pd.read_csv("/content/poems-100.csv")

# Ignore first row and extract poems
poems = df.iloc[:, 0].astype(str).tolist()

# Combine all poems into one text corpus
text_data = " ".join(poems).lower()

print("Number of poems:", len(poems))
print("Sample text:\n", text_data[:500])

Number of poems: 100
Sample text:
 o my luve's like a red, red rose
that’s newly sprung in june;
o my luve's like the melodie
that’s sweetly play'd in tune.

as fair art thou, my bonnie lass,
so deep in luve am i:
and i will luve thee still, my dear,
till a’ the seas gang dry:

till a’ the seas gang dry, my dear,
and the rocks melt wi’ the sun:
i will luve thee still, my dear,
while the sands o’ life shall run.

and fare thee well, my only luve
and fare thee well, a while!
and i will come again, my luve,
tho’ it were ten thousand


In [7]:
# Simple whitespace tokenization
tokens = text_data.split()

# Build vocabulary
word_counts = Counter(tokens)
vocab = sorted(word_counts.keys())
vocab_size = len(vocab)

# Word to index mapping
word2idx = {word: i for i, word in enumerate(vocab)}
idx2word = {i: word for word, i in word2idx.items()}

print("Vocabulary size:", vocab_size)
print("Sample vocab words:", vocab[:20])

Vocabulary size: 6989
Sample vocab words: ['"he', '"most', '"oh,', "'greatly", "'neath", "'our", "'s", "'tis", "'twas", "'twere", "'twill", "('tis", '(1)', '(and', '(as', '(behind', '(come', '(even', '(floating', '(for']


In [8]:

class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_size = hidden_size

        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Why = np.random.randn(output_size, hidden_size) * 0.01

        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs):
        h = np.zeros((self.hidden_size, 1))
        outputs = []

        for x in inputs:
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)
            y = np.dot(self.Why, h) + self.by
            outputs.append(y)

        return outputs, h


In [9]:

sequence_length = 5

def create_sequences(tokens, seq_length):
    sequences = []
    for i in range(len(tokens) - seq_length):
        seq = tokens[i:i+seq_length]
        target = tokens[i+seq_length]
        sequences.append((seq, target))
    return sequences

sequences = create_sequences(tokens, sequence_length)
print("Total sequences:", len(sequences))


Total sequences: 24729


In [10]:
def one_hot_encode(word, vocab_size):
    vec = np.zeros(vocab_size)
    vec[word2idx[word]] = 1
    return vec

In [11]:
class OneHotDataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq, target = self.sequences[idx]

        x = [one_hot_encode(word, vocab_size) for word in seq]
        x = torch.tensor(x, dtype=torch.float32)

        y = torch.tensor(word2idx[target], dtype=torch.long)

        return x, y

dataset_onehot = OneHotDataset(sequences)
loader_onehot = torch.utils.data.DataLoader(dataset_onehot, batch_size=16, shuffle=True)

In [12]:
class RNN_OneHot(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN_OneHot, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

In [13]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [14]:
model_onehot = RNN_OneHot(vocab_size, 128, vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_onehot.parameters(), lr=0.001)

epochs = 10
start_time = time.time()

for epoch in range(epochs):
    total_loss = 0
    for x, y in loader_onehot:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        output = model_onehot(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader_onehot):.4f}")

print("Training Time:", time.time() - start_time)

/tmp/ipython-input-76012416.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  x = torch.tensor(x, dtype=torch.float32)


Epoch 1, Loss: 7.2562
Epoch 2, Loss: 6.4333
Epoch 3, Loss: 5.7067
Epoch 4, Loss: 4.8471
Epoch 5, Loss: 3.9490
Epoch 6, Loss: 3.0981
Epoch 7, Loss: 2.3498
Epoch 8, Loss: 1.7197
Epoch 9, Loss: 1.2370
Epoch 10, Loss: 0.8921
Training Time: 1184.5059220790863


In [15]:
def generate_text_onehot(model, start_text, length=20):
    model.eval()
    words = start_text.lower().split()

    for _ in range(length):
        seq = words[-sequence_length:]
        x = [one_hot_encode(word, vocab_size) for word in seq]
        x = torch.tensor([x], dtype=torch.float32).to(device)

        with torch.no_grad():
            output = model(x)
            predicted = torch.argmax(output, dim=1).item()

        next_word = idx2word[predicted]
        words.append(next_word)

    return " ".join(words)

print(generate_text_onehot(model_onehot, "the night was"))

the night was shine men, the lips. that hangs blood the wife's or altamahaw, twelve by the supper with night, light sings and


In [16]:
class IndexedDataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq, target = self.sequences[idx]
        x = torch.tensor([word2idx[word] for word in seq], dtype=torch.long)
        y = torch.tensor(word2idx[target], dtype=torch.long)
        return x, y

dataset_embed = IndexedDataset(sequences)
loader_embed = torch.utils.data.DataLoader(dataset_embed, batch_size=64, shuffle=True)

In [17]:

class RNN_Embedding(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size):
        super(RNN_Embedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

In [18]:
model_embed = RNN_Embedding(vocab_size, 100, 128).to(device)
optimizer = optim.Adam(model_embed.parameters(), lr=0.001)

start_time = time.time()

for epoch in range(epochs):
    total_loss = 0
    for x, y in loader_embed:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        output = model_embed(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader_embed):.4f}")

print("Training Time:", time.time() - start_time)

Epoch 1, Loss: 7.3467
Epoch 2, Loss: 6.4326
Epoch 3, Loss: 5.9048
Epoch 4, Loss: 5.3831
Epoch 5, Loss: 4.8532
Epoch 6, Loss: 4.3287
Epoch 7, Loss: 3.8164
Epoch 8, Loss: 3.3375
Epoch 9, Loss: 2.9092
Epoch 10, Loss: 2.5275
Training Time: 73.1105010509491


In [19]:
def generate_text_embedding(model, start_text, length=20):
    model.eval()
    words = start_text.lower().split()

    for _ in range(length):
        seq = words[-sequence_length:]
        x = torch.tensor([[word2idx[word] for word in seq]], dtype=torch.long).to(device)

        with torch.no_grad():
            output = model(x)
            predicted = torch.argmax(output, dim=1).item()

        next_word = idx2word[predicted]
        words.append(next_word)

    return " ".join(words)

print(generate_text_embedding(model_embed, "the night was"))

the night was the reason of the stairs, of the earth, and of the house of the stairs, of the earth, and of
